In [1]:
%pylab inline
import scipy.stats as stats
import astropy.stats as astats
import numpy.random as random

Populating the interactive namespace from numpy and matplotlib


In [3]:
import hlmean

In [5]:
def mode2(data,**kwargs):
# note: provide bins and (optionally) range keywords to not use
# defaults of np.histogram (10 bins, full range)
    counts,edges=np.histogram(data,**kwargs)
    whmax=np.argmax(counts)
    mode=(edges[whmax]+edges[whmax+1])/2
    return(mode)

# Q1, Part A
### Monte Carlo setup

Foutlier = 0, ndata = 100 (perfect Gaussian)

In [87]:
nsims=int(1E4) # reduced from 5E4 due to some functions not being vectorized, eg hlmean. Still sufficiently large number of simulations.
ndata=100
foutlier=0

isoutlier=random.rand(nsims,ndata) < foutlier

fakedata=(1-isoutlier)*(random.randn(nsims,ndata)*930.+3150) \
+ (isoutlier)*(random.randn(nsims,ndata)*200.+4750)

In [88]:
means = np.mean(fakedata, axis = 1)
medians = np.median(fakedata, axis = 1)
modes = [mode2(fakedata0,bins=np.arange(fakedata0.min(),fakedata0.max(),50)) for fakedata0 in fakedata]
modes = np.array(modes)
hlmeans = [hlmean.hlmean(fakedata0) for fakedata0 in fakedata]
hlmeans = np.array(hlmeans)
tmeans = [stats.tmean(fakedata0,limits=np.percentile(fakedata0,(10,90))) for fakedata0 in fakedata]
tmeans = np.array(tmeans)
sclips = []
for i in range(0, nsims):
    clipped_data,low_threshold,high_threshold =  stats.sigmaclip(fakedata[i],low=3.,high=3.)
    sclips += [np.mean(clipped_data)]
sclips = np.array(sclips)
winsors = [np.mean(stats.mstats.winsorize(fakedata0,limits=(.1,.1))) for fakedata0 in fakedata]
winsors = np.array(winsors)
biweights = astats.biweight_location(fakedata, axis = 1)

In [89]:
print("Expected mean: 3150")
print()
print("BIAS OF ESTIMATORS:")
print(f'Mean: {np.mean(means - 3150):.4f}')
print(f'Median: {np.mean(medians - 3150):.4f}')
print(f'Mode: {np.mean(modes - 3150):.4f}')
print(f'H-L mean: {np.mean(hlmeans - 3150):.4f}')
print(f'10% Trimmed mean: {np.mean(tmeans - 3150):.4f}')
print(f'Sigma-clipped: {np.mean(sclips - 3150):.4f}')
print(f'Winsorized: {np.mean(winsors - 3150):.4f}')
print(f'Biweight: {np.mean(biweights - 3150):.4f}')
print()
print("SPREAD OF ESTIMATORS:")
print(f'Mean: {np.std(means):.4f}')
print(f'Median: {np.std(medians):.4f}')
print(f'Mode: {np.std(modes):.4f}')
print(f'H-L mean: {np.std(hlmeans):.4f}')
print(f'10% Trimmed mean: {np.std(tmeans):.4f}')
print(f'Sigma-clipped: {np.std(sclips):.4f}')
print(f'Winsorized: {np.std(winsors):.4f}')
print(f'Biweight: {np.std(biweights):.4f}')

Expected mean: 3150

BIAS OF ESTIMATORS:
Mean: 0.5778
Median: -0.1513
Mode: -170.0752
H-L mean: 0.6586
10% Trimmed mean: 0.5588
Sigma-clipped: 0.6868
Winsorized: 0.7596
Biweight: 0.2509

SPREAD OF ESTIMATORS:
Mean: 93.4153
Median: 115.7042
Mode: 488.8806
H-L mean: 96.3031
10% Trimmed mean: 96.1619
Sigma-clipped: 94.6896
Winsorized: 95.1428
Biweight: 99.8122


In the case of perfect Gaussian data, if we seek to minimize bias, then Biweight and Median are both good choices, Median being the least biased choice. However, Median is the second highest for spread, and Biweight's spread is lower but still higher than the mean's spread, which is the lowest. Thus, Biweight serves as a compromise between mean and median. Trimmed mean also is less biased than the mean, but not significantly so, and this advantage comes at the cost of a higher spread. All the other statistics are more biased than the mean and have a higher spread. 

# Q1, Part B
### Monte Carlo setup

Foutlier = 0.1, ndata = 5

In [77]:
nsims=int(1E4) # reduced from 5E4 due to some functions not being vectorized, eg hlmean. Still sufficiently large number of simulations.
ndata=5
foutlier=0.1

isoutlier=random.rand(nsims,ndata) < foutlier

fakedata=(1-isoutlier)*(random.randn(nsims,ndata)*930.+3150) \
+ (isoutlier)*(random.randn(nsims,ndata)*200.+4750)

In [78]:
means = np.mean(fakedata, axis = 1)
medians = np.median(fakedata, axis = 1)
modes = [mode2(fakedata0,bins=np.arange(fakedata0.min(),fakedata0.max(),50)) for fakedata0 in fakedata]
modes = np.array(modes)
hlmeans = [hlmean.hlmean(fakedata0) for fakedata0 in fakedata]
hlmeans = np.array(hlmeans)
tmeans = [stats.tmean(fakedata0,limits=np.percentile(fakedata0,(10,90))) for fakedata0 in fakedata]
tmeans = np.array(tmeans)
sclips = []
for i in range(0, nsims):
    clipped_data,low_threshold,high_threshold =  stats.sigmaclip(fakedata[i],low=3.,high=3.)
    sclips += [np.mean(clipped_data)]
sclips = np.array(sclips)
winsors = [np.mean(stats.mstats.winsorize(fakedata0,limits=(.1,.1))) for fakedata0 in fakedata]
winsors = np.array(winsors)
biweights = astats.biweight_location(fakedata, axis = 1)

In [79]:
print("Expected mean: 3150")
print()
print("BIAS OF ESTIMATORS:")
print(f'Mean: {np.mean(means - 3150):.4f}')
print(f'Median: {np.mean(medians - 3150):.4f}')
print(f'Mode: {np.mean(modes - 3150):.4f}')
print(f'H-L mean: {np.mean(hlmeans - 3150):.4f}')
print(f'10% Trimmed mean: {np.mean(tmeans - 3150):.4f}')
print(f'Sigma-clipped: {np.mean(sclips - 3150):.4f}')
print(f'Winsorized: {np.mean(winsors - 3150):.4f}')
print(f'Biweight: {np.mean(biweights - 3150):.4f}')
print()
print("SPREAD OF ESTIMATORS:")
print(f'Mean: {np.std(means):.4f}')
print(f'Median: {np.std(medians):.4f}')
print(f'Mode: {np.std(modes):.4f}')
print(f'H-L mean: {np.std(hlmeans):.4f}')
print(f'10% Trimmed mean: {np.std(tmeans):.4f}')
print(f'Sigma-clipped: {np.std(sclips):.4f}')
print(f'Winsorized: {np.std(winsors):.4f}')
print(f'Biweight: {np.std(biweights):.4f}')

Expected mean: 3150

BIAS OF ESTIMATORS:
Mean: 160.4358
Median: 156.6664
Mode: -919.9316
H-L mean: 161.7034
10% Trimmed mean: 164.1139
Sigma-clipped: 160.4358
Winsorized: 160.4358
Biweight: 161.8702

SPREAD OF ESTIMATORS:
Mean: 452.5797
Median: 578.7716
Mode: 713.2324
H-L mean: 482.1006
10% Trimmed mean: 504.4914
Sigma-clipped: 452.5797
Winsorized: 452.5797
Biweight: 549.6673


In this case, we should use the mean, since it has the second lowest bias (after the the median) and the lowest spread. Sigma-clipped and winsorized provide no additional benefit over the regular mean in this instance, since they all have the same bias and spread (nothing is clipped or winsorized in the case of such a small sample). 

### Monte Carlo setup

Foutlier = 0.1, ndata = 25

In [80]:
nsims=int(1E4) # reduced from 5E4 due to some functions not being vectorized, eg hlmean. Still sufficiently large number of simulations.
ndata=100
foutlier=0.1

isoutlier=random.rand(nsims,ndata) < foutlier

fakedata=(1-isoutlier)*(random.randn(nsims,ndata)*930.+3150) \
+ (isoutlier)*(random.randn(nsims,ndata)*200.+4750)

In [81]:
means = np.mean(fakedata, axis = 1)
medians = np.median(fakedata, axis = 1)
modes = [mode2(fakedata0,bins=np.arange(fakedata0.min(),fakedata0.max(),50)) for fakedata0 in fakedata]
modes = np.array(modes)
hlmeans = [hlmean.hlmean(fakedata0) for fakedata0 in fakedata]
hlmeans = np.array(hlmeans)
tmeans = [stats.tmean(fakedata0,limits=np.percentile(fakedata0,(10,90))) for fakedata0 in fakedata]
tmeans = np.array(tmeans)
sclips = []
for i in range(0, nsims):
    clipped_data,low_threshold,high_threshold =  stats.sigmaclip(fakedata[i],low=3.,high=3.)
    sclips += [np.mean(clipped_data)]
sclips = np.array(sclips)
winsors = [np.mean(stats.mstats.winsorize(fakedata0,limits=(.1,.1))) for fakedata0 in fakedata]
winsors = np.array(winsors)
biweights = astats.biweight_location(fakedata, axis = 1)

In [82]:
print("Expected mean: 3150")
print()
print("BIAS OF ESTIMATORS:")
print(f'Mean: {np.mean(means - 3150):.4f}')
print(f'Median: {np.mean(medians - 3150):.4f}')
print(f'Mode: {np.mean(modes - 3150):.4f}')
print(f'H-L mean: {np.mean(hlmeans - 3150):.4f}')
print(f'10% Trimmed mean: {np.mean(tmeans - 3150):.4f}')
print(f'Sigma-clipped: {np.mean(sclips - 3150):.4f}')
print(f'Winsorized: {np.mean(winsors - 3150):.4f}')
print(f'Biweight: {np.mean(biweights - 3150):.4f}')
print()
print("SPREAD OF ESTIMATORS:")
print(f'Mean: {np.std(means):.4f}')
print(f'Median: {np.std(medians):.4f}')
print(f'Mode: {np.std(modes):.4f}')
print(f'H-L mean: {np.std(hlmeans):.4f}')
print(f'10% Trimmed mean: {np.std(tmeans):.4f}')
print(f'Sigma-clipped: {np.std(sclips):.4f}')
print(f'Winsorized: {np.std(winsors):.4f}')
print(f'Biweight: {np.std(biweights):.4f}')

Expected mean: 3150

BIAS OF ESTIMATORS:
Mean: 160.3033
Median: 132.4807
Mode: -88.7331
H-L mean: 162.0580
10% Trimmed mean: 169.3545
Sigma-clipped: 162.0528
Winsorized: 177.4923
Biweight: 157.4129

SPREAD OF ESTIMATORS:
Mean: 101.2232
Median: 132.0090
Mode: 611.0597
H-L mean: 108.8471
10% Trimmed mean: 110.3218
Sigma-clipped: 101.9459
Winsorized: 103.9501
Biweight: 112.1591


In this case, we should use the median, since it has the second lowest bias (after mode) and only slightly higher spread than the mean. The mode, while least biased in this instance, has a much higher spread. While biweight is also less biased than the mean, it is more biased than the median, and has a higher spread than the mean.

### Monte Carlo setup

Foutlier = 0.1, ndata = 100

In [83]:
nsims=int(1E4) # reduced from 5E4 due to some functions not being vectorized, eg hlmean. Still sufficiently large number of simulations.
ndata=25
foutlier=0.1

isoutlier=random.rand(nsims,ndata) < foutlier

fakedata=(1-isoutlier)*(random.randn(nsims,ndata)*930.+3150) \
+ (isoutlier)*(random.randn(nsims,ndata)*200.+4750)

In [84]:
means = np.mean(fakedata, axis = 1)
medians = np.median(fakedata, axis = 1)
modes = [mode2(fakedata0,bins=np.arange(fakedata0.min(),fakedata0.max(),50)) for fakedata0 in fakedata]
modes = np.array(modes)
hlmeans = [hlmean.hlmean(fakedata0) for fakedata0 in fakedata]
hlmeans = np.array(hlmeans)
tmeans = [stats.tmean(fakedata0,limits=np.percentile(fakedata0,(10,90))) for fakedata0 in fakedata]
tmeans = np.array(tmeans)
sclips = []
for i in range(0, nsims):
    clipped_data,low_threshold,high_threshold =  stats.sigmaclip(fakedata[i],low=3.,high=3.)
    sclips += [np.mean(clipped_data)]
sclips = np.array(sclips)
winsors = [np.mean(stats.mstats.winsorize(fakedata0,limits=(.1,.1))) for fakedata0 in fakedata]
winsors = np.array(winsors)
biweights = astats.biweight_location(fakedata, axis = 1)

In [85]:
print("Expected mean: 3150")
print()
print("BIAS OF ESTIMATORS:")
print(f'Mean: {np.mean(means - 3150):.4f}')
print(f'Median: {np.mean(medians - 3150):.4f}')
print(f'Mode: {np.mean(modes - 3150):.4f}')
print(f'H-L mean: {np.mean(hlmeans - 3150):.4f}')
print(f'10% Trimmed mean: {np.mean(tmeans - 3150):.4f}')
print(f'Sigma-clipped: {np.mean(sclips - 3150):.4f}')
print(f'Winsorized: {np.mean(winsors - 3150):.4f}')
print(f'Biweight: {np.mean(biweights - 3150):.4f}')
print()
print("SPREAD OF ESTIMATORS:")
print(f'Mean: {np.std(means):.4f}')
print(f'Median: {np.std(medians):.4f}')
print(f'Mode: {np.std(modes):.4f}')
print(f'H-L mean: {np.std(hlmeans):.4f}')
print(f'10% Trimmed mean: {np.std(tmeans):.4f}')
print(f'Sigma-clipped: {np.std(sclips):.4f}')
print(f'Winsorized: {np.std(winsors):.4f}')
print(f'Biweight: {np.std(biweights):.4f}')

Expected mean: 3150

BIAS OF ESTIMATORS:
Mean: 158.4455
Median: 134.9621
Mode: -346.6831
H-L mean: 160.4018
10% Trimmed mean: 164.6754
Sigma-clipped: 159.0059
Winsorized: 170.0312
Biweight: 153.6898

SPREAD OF ESTIMATORS:
Mean: 201.9668
Median: 261.3799
Mode: 747.7605
H-L mean: 215.9766
10% Trimmed mean: 219.8671
Sigma-clipped: 202.5383
Winsorized: 206.8125
Biweight: 225.8987


In this case, the median again proves to be the best due to having the least bias and a comparable, if high, spread. As with the previous case, the mean has a much lower spread, but it is more biased. The biweight mean strikes a compromise between mean and median, erring on the side of more bias and less spread. 

# Q2, Part A

In [145]:
nsims=int(1E4)
ndata=100
foutlier=0

isoutlier=random.rand(nsims,ndata) < foutlier

fakedata=(1-isoutlier)*(random.randn(nsims,ndata)*930.+3150) \
+ (isoutlier)*(random.uniform(low = 0, high = 6500, size = (nsims,ndata))*200.+4750)

In [146]:
stds = np.std(fakedata, axis = 1)
normmeanabsdevs = np.mean(np.abs(fakedata-np.reshape(np.mean(fakedata, axis = 1), (nsims, 1))), axis = 1)/0.7979
normmads = np.median(np.abs(fakedata-np.reshape(np.median(fakedata, axis = 1), (nsims, 1))), axis = 1)/0.6745
bimidvars = astats.biweight_midvariance(fakedata, axis = 1)
iqrs = (np.percentile(fakedata, 75, axis = 1) - np.percentile(fakedata, 25, axis = 1))/1.349
tstds = np.array([stats.tstd(fakedata0, limits=(np.percentile(fakedata0, 10),np.percentile(fakedata0, 90))) for fakedata0 in fakedata])

In [147]:
print("Expected standard deviation: 930")
print()
print("BIAS OF ESTIMATORS:")
print(f'Standard deviation: {np.mean(stds - 930):.4f}')
print(f'Average absolute deviation: {np.mean(normmeanabsdevs - 930):.4f}')
print(f'MAD: {np.mean(normmads - 930):.4f}')
print(f'Biweight standard deviation: {np.mean(np.sqrt(bimidvars) - 930):.4f}')
print(f'IQR: {np.mean(iqrs - 930):.4f}')
print(f'10% Trimmed standard deviation: {np.mean(tstds - 930):.4f}')
print()
print("Spread OF ESTIMATORS:")
print(f'Standard deviation: {np.std(stds):.4f}')
print(f'Average absolute deviation: {np.std(normmeanabsdevs):.4f}')
print(f'MAD: {np.std(normmads):.4f}')
print(f'Biweight standard deviation: {np.std(np.sqrt(bimidvars)):.4f}')
print(f'IQR: {np.std(iqrs):.4f}')
print(f'10% Trimmed standard deviation: {np.std(tstds):.4f}')

Expected standard deviation: 930

BIAS OF ESTIMATORS:
Standard deviation: -6.2034
Average absolute deviation: -4.0455
MAD: -5.8634
Biweight standard deviation: 3.2727
IQR: -11.5846
10% Trimmed standard deviation: -309.1215

Spread OF ESTIMATORS:
Standard deviation: 65.4496
Average absolute deviation: 70.1123
MAD: 108.5375
Biweight standard deviation: 69.0927
IQR: 107.3595
10% Trimmed standard deviation: 55.4116


In this Gaussian case, the Biweight standard deviation has the lowest bias and a fairly similar spread to the standard deviation, making it a good estimator. The average abs standard deviation also outperforms the regular standard deviation when it comes to bias, at the cost of spread. The trimmed standard deviation has a low spread but it is very biased.

# Q2, Part B

In [148]:
nsims=int(1E4)
ndata=5
foutlier=0.1

isoutlier=random.rand(nsims,ndata) < foutlier

fakedata=(1-isoutlier)*(random.randn(nsims,ndata)*930.+3150) \
+ (isoutlier)*(random.uniform(low = 0, high = 6500, size = (nsims,ndata)))

In [149]:
stds = np.std(fakedata, axis = 1)
normmeanabsdevs = np.mean(np.abs(fakedata-np.reshape(np.mean(fakedata, axis = 1), (nsims, 1))), axis = 1)/0.7979
normmads = np.median(np.abs(fakedata-np.reshape(np.median(fakedata, axis = 1), (nsims, 1))), axis = 1)/0.6745
bimidvars = astats.biweight_midvariance(fakedata, axis = 1)
iqrs = (np.percentile(fakedata, 75, axis = 1) - np.percentile(fakedata, 25, axis = 1))/1.349
tstds = np.array([stats.tstd(fakedata0, limits=(np.percentile(fakedata0, 10),np.percentile(fakedata0, 90))) for fakedata0 in fakedata])

In [150]:
print("Expected standard deviation: 930")
print()
print("BIAS OF ESTIMATORS:")
print(f'Standard deviation: {np.mean(stds - 930):.4f}')
print(f'Average absolute deviation: {np.mean(normmeanabsdevs - 930):.4f}')
print(f'MAD: {np.mean(normmads - 930):.4f}')
print(f'Biweight standard deviation: {np.mean(np.sqrt(bimidvars) - 930):.4f}')
print(f'IQR: {np.mean(iqrs - 930):.4f}')
print(f'10% Trimmed standard deviation: {np.mean(tstds - 930):.4f}')
print()
print("Spread OF ESTIMATORS:")
print(f'Standard deviation: {np.std(stds):.4f}')
print(f'Average absolute deviation: {np.std(normmeanabsdevs):.4f}')
print(f'MAD: {np.std(normmads):.4f}')
print(f'Biweight standard deviation: {np.std(np.sqrt(bimidvars)):.4f}')
print(f'IQR: {np.std(iqrs):.4f}')
print(f'10% Trimmed standard deviation: {np.std(tstds):.4f}')

Expected standard deviation: 930

BIAS OF ESTIMATORS:
Standard deviation: -39.6606
Average absolute deviation: 12.5395
MAD: -76.9644
Biweight standard deviation: -71.2565
IQR: -177.2946
10% Trimmed standard deviation: -396.2571

Spread OF ESTIMATORS:
Standard deviation: 343.4008
Average absolute deviation: 368.8269
MAD: 511.2933
Biweight standard deviation: 409.9788
IQR: 439.2326
10% Trimmed standard deviation: 311.5564


In this case, the average absolute deviation has a far lower bias, so it is the best choice for measuring velocity dispersion, even if the standard deviation and trimmed std have a slightly lower spread.

In [151]:
nsims=int(1E4)
ndata=25
foutlier=0.1

isoutlier=random.rand(nsims,ndata) < foutlier

fakedata=(1-isoutlier)*(random.randn(nsims,ndata)*930.+3150) \
+ (isoutlier)*(random.uniform(low = 0, high = 6500, size = (nsims,ndata)))

In [152]:
stds = np.std(fakedata, axis = 1)
normmeanabsdevs = np.mean(np.abs(fakedata-np.reshape(np.mean(fakedata, axis = 1), (nsims, 1))), axis = 1)/0.7979
normmads = np.median(np.abs(fakedata-np.reshape(np.median(fakedata, axis = 1), (nsims, 1))), axis = 1)/0.6745
bimidvars = astats.biweight_midvariance(fakedata, axis = 1)
iqrs = (np.percentile(fakedata, 75, axis = 1) - np.percentile(fakedata, 25, axis = 1))/1.349
tstds = np.array([stats.tstd(fakedata0, limits=(np.percentile(fakedata0, 10),np.percentile(fakedata0, 90))) for fakedata0 in fakedata])

In [153]:
print("Expected standard deviation: 930")
print()
print("BIAS OF ESTIMATORS:")
print(f'Standard deviation: {np.mean(stds - 930):.4f}')
print(f'Average absolute deviation: {np.mean(normmeanabsdevs - 930):.4f}')
print(f'MAD: {np.mean(normmads - 930):.4f}')
print(f'Biweight standard deviation: {np.mean(np.sqrt(bimidvars) - 930):.4f}')
print(f'IQR: {np.mean(iqrs - 930):.4f}')
print(f'10% Trimmed standard deviation: {np.mean(tstds - 930):.4f}')
print()
print("Spread OF ESTIMATORS:")
print(f'Standard deviation: {np.std(stds):.4f}')
print(f'Average absolute deviation: {np.std(normmeanabsdevs):.4f}')
print(f'MAD: {np.std(normmads):.4f}')
print(f'Biweight standard deviation: {np.std(np.sqrt(bimidvars)):.4f}')
print(f'IQR: {np.std(iqrs):.4f}')
print(f'10% Trimmed standard deviation: {np.std(tstds):.4f}')

Expected standard deviation: 930

BIAS OF ESTIMATORS:
Standard deviation: 97.8614
Average absolute deviation: 89.5477
MAD: 40.9750
Biweight standard deviation: 92.7675
IQR: 15.5951
10% Trimmed standard deviation: -283.2803

Spread OF ESTIMATORS:
Standard deviation: 162.6711
Average absolute deviation: 167.9657
MAD: 237.1376
Biweight standard deviation: 175.5518
IQR: 229.9250
10% Trimmed standard deviation: 127.1450


In this case, we see the IQR outperform the all others in bias with a somewhat higher, but not unreasonable spread, making it the best choice. The MAD takes second place for bias, but the trimmed standard deviation has the lowest spread (at the cost of large bias).

In [154]:
nsims=int(1E4)
ndata=100
foutlier=0.1

isoutlier=random.rand(nsims,ndata) < foutlier

fakedata=(1-isoutlier)*(random.randn(nsims,ndata)*930.+3150) \
+ (isoutlier)*(random.uniform(low = 0, high = 6500, size = (nsims,ndata)))

In [155]:
stds = np.std(fakedata, axis = 1)
normmeanabsdevs = np.mean(np.abs(fakedata-np.reshape(np.mean(fakedata, axis = 1), (nsims, 1))), axis = 1)/0.7979
normmads = np.median(np.abs(fakedata-np.reshape(np.median(fakedata, axis = 1), (nsims, 1))), axis = 1)/0.6745
bimidvars = astats.biweight_midvariance(fakedata, axis = 1)
iqrs = (np.percentile(fakedata, 75, axis = 1) - np.percentile(fakedata, 25, axis = 1))/1.349
tstds = np.array([stats.tstd(fakedata0, limits=(np.percentile(fakedata0, 10),np.percentile(fakedata0, 90))) for fakedata0 in fakedata])

In [156]:
print("Expected standard deviation: 930")
print()
print("BIAS OF ESTIMATORS:")
print(f'Standard deviation: {np.mean(stds - 930):.4f}')
print(f'Average absolute deviation: {np.mean(normmeanabsdevs - 930):.4f}')
print(f'MAD: {np.mean(normmads - 930):.4f}')
print(f'Biweight standard deviation: {np.mean(np.sqrt(bimidvars) - 930):.4f}')
print(f'IQR: {np.mean(iqrs - 930):.4f}')
print(f'10% Trimmed standard deviation: {np.mean(tstds - 930):.4f}')
print()
print("Spread OF ESTIMATORS:")
print(f'Standard deviation: {np.std(stds):.4f}')
print(f'Average absolute deviation: {np.std(normmeanabsdevs):.4f}')
print(f'MAD: {np.std(normmads):.4f}')
print(f'Biweight standard deviation: {np.std(np.sqrt(bimidvars)):.4f}')
print(f'IQR: {np.std(iqrs):.4f}')
print(f'10% Trimmed standard deviation: {np.std(tstds):.4f}')

Expected standard deviation: 930

BIAS OF ESTIMATORS:
Standard deviation: 123.9276
Average absolute deviation: 105.1157
MAD: 64.7894
Biweight standard deviation: 115.6268
IQR: 57.6636
10% Trimmed standard deviation: -254.4174

Spread OF ESTIMATORS:
Standard deviation: 81.6178
Average absolute deviation: 83.2395
MAD: 116.7903
Biweight standard deviation: 86.4769
IQR: 114.9485
10% Trimmed standard deviation: 62.1024


Here too, we see the IQR outperform the all others in bias with a somewhat higher, but not unreasonable spread, making it the best choice. The MAD again takes second place for bias, and the trimmed standard deviation has the lowest spread (at the cost of large bias).